<a href="https://colab.research.google.com/github/L-4-r-s/MADE-Tensorflow2/blob/main/Final_MADE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mandatory imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
import time
from keras import backend as k
from keras import metrics
from tensorflow.keras import activations
from tensorflow.keras.layers import Input, Layer
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.datasets import mnist

Load Dataset ( https://github.com/mgermain/MADE/releases/download/ICML2015/binarized_mnist.npz )

In [ ]:
# Load Mnist Data used for FedWeIT experiments

def _generate_mnist():
    global X_train, X_valid, X_test 
    seprate_ratio = (5/7, 1/7, 1/7)
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    x_temp = np.concatenate((X_train, X_test))
    x = x_temp.reshape(x_temp.shape[0],x_temp.shape[1]*x_temp.shape[2]) #flatten 28x28 pixels to one dimension (784 inputs)
    x = np.where(x > 127, 1, 0) #binarize x
    y = np.concatenate((Y_train, Y_test))
    
    idx_shuffled = np.arange(len(x))
    random.seed(777)
    random.shuffle(idx_shuffled)
    labels = [5,8,7]
    
    idx = np.concatenate([np.where(y[:]==c)[0] for c in labels], axis=0)
    #shuffle so samples are not sampled by class cause of previous concat operation
    random.seed(777)
    random.shuffle(idx) # shuffle order of training samples derived for current task
    x_task = x[idx]

    num_examples = len(x_task)
    num_train = int(num_examples*seprate_ratio[0])
    num_test = int(num_examples*seprate_ratio[1])
    num_valid = num_examples - num_train - num_test
    X_train = x_task[:num_train]
    X_valid = x_task[num_train+num_test:]
    X_test = x_task[num_train:num_train+num_test]

_generate_mnist()

In [2]:
# I stored the dataset in my drive
data_path = 'drive/MyDrive/binarized_mnist.npz'
binarized_mnist = np.load(data_path)
X_train, X_valid, X_test = binarized_mnist['train_data'], binarized_mnist['valid_data'], binarized_mnist['test_data']
print(f"train dataset shape: {X_train.shape}, train dataset shape: {X_test.shape}, train dataset shape = {X_valid.shape}")

train dataset shape: (50000, 784), train dataset shape: (10000, 784), train dataset shape = (10000, 784)


Create Mask Generator Module for creating/managing MADEs masks

In [3]:
class MaskGenerator(object):
  # num_masks: The amount of masks that will be cycled through during training. if num_masks == 1 then connectivity agnostic training is disabled
  # units_per_layer = Array containing # of units per layer
  # seed = The seed used for randomly sampling the masks, for guaranteeing reproducability
  # natural_input_order = Boolean defining if the natural input order (x1, x2, x3 etc) should be used
  # current_mask: Integer to keep track of the mask currently used (xth mask)
  # m: The mask values assigned to the networks units. 0 is the index of the input layer, 1 is the index of the first hidden layer and so on
  def __init__(self, num_masks, units_per_layer, natural_input_order = False, seed=42):
    self.num_masks = num_masks
    self.units_per_layer = units_per_layer
    self.seed = seed
    self.natural_input_order = natural_input_order
    self.current_mask = 0
    self.m = {}

    if natural_input_order: # init input ordering according to settings
      self.m[0] = np.arange(self.units_per_layer[0])
    else:
      self.shuffle_inputs(return_mask = False)
  
  #Iterate through the hidden layers, resample new connectivity values m and build/return the resulting new masks
  def shuffle_masks(self):
    layer_amount = len(self.units_per_layer)
    rng = np.random.RandomState(self.seed+self.current_mask)
    self.current_mask = (self.current_mask + 1) % self.num_masks # Cycle through masks
    for i in range(1, layer_amount -1): #skip input layer & output layer and only iterate through hidden_layers
      self.m[i] = rng.randint(self.m[i-1].min(), self.units_per_layer[0] -1, size = self.units_per_layer[i]) # sample m from [min_m(previous_layer, d-1)] for all hidden units
    new_masks = [tf.convert_to_tensor((self.m[l-1][:, None] <= self.m[l][None,:]), dtype=np.float32) for l in range(1, layer_amount-1)] # build hidden layer masks
    new_masks.append(tf.convert_to_tensor((self.m[layer_amount-2][:, None] < self.m[0][None, :]), dtype = np.float32)) #build output layer mask. Note that the m values for the output layer are the same as for the input layer 
    return new_masks

  # builds & returns direct mask. Call this method after shuffling inputs if order_agnostic training is active.
  # Note that the Mask values m are the same for both input and output layers
  def get_direct_mask(self):
    return tf.convert_to_tensor((self.m[0][:, None] < self.m[0][None, :]), dtype = np.float32)

  # shuffle input ordering and return new mask for first hidden layer
  def shuffle_inputs(self, return_mask = True):
    self.m[0] = np.random.permutation(self.units_per_layer[0])
    if return_mask:
      return tf.convert_to_tensor((self.m[0][:, None] <= self.m[1][None,:]), dtype=np.float32)
    return

Custom Layer for MADE masking

In [4]:
# should be self explaining
class MaskedLayer(Layer):
    def __init__(self,
                units,
                mask,
                activation='relu',
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                **kwargs):
      self.units = units
      self.mask = mask
      self.activation = activations.get(activation)
      self.kernel_initializer = kernel_initializer
      self.bias_initializer = bias_initializer
      super(MaskedLayer, self).__init__(**kwargs)

    def build(self, input_shape):
      #self.input_dim = input_shape[-1] if self.x_dim is None else input_shape[0][-1]

      self.W = self.add_weight(shape=self.mask.shape,
                                  initializer=self.kernel_initializer,
                                  name='W')

      self.bias = self.add_weight(shape=(self.units,),
                                      initializer=self.bias_initializer,
                                      name='bias')

      self.built = True

    def call(self, inputs):
        ## Modified keras.Dense to account for the mask
        masked_weights = self.W*self.mask
        output = k.dot(inputs, masked_weights)
        output = k.bias_add(output, self.bias, data_format = 'channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output

    def set_mask(self, mask):
        self.mask = mask

    def get_mask(self):
        return self.mask

    def compute_output_shape(self, input_shape):
        ##Same as keras.Dense
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)



class ConditionningMaskedLayer(MaskedLayer):
    def __init__(self, 
                units,
                mask,
                activation='relu',
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                use_cond_mask=False,
                **kwargs):
        self.use_cond_mask = use_cond_mask
        super(ConditionningMaskedLayer, self).__init__(units,
                mask,
                activation,
                kernel_initializer,
                bias_initializer, **kwargs)

    def build(self, input_shape):
        if self.use_cond_mask:
            self.U = self.add_weight(shape=self.mask.shape,
                                     initializer=self.kernel_initializer,
                                     name='U')
        super().build(input_shape)

    def call(self, inputs):
        if self.use_cond_mask == False:
          return super().call(inputs)
        masked_w_weights = self.W*self.mask
        masked_u_weights_times_one_vec = k.dot(tf.ones(tf.shape(inputs)),self.U*self.mask)
        weighted_input = k.dot(inputs, masked_w_weights)
        weighted_input_and_bias = k.bias_add(weighted_input, self.bias, data_format = 'channels_last')
        output = weighted_input_and_bias + masked_u_weights_times_one_vec
        if self.activation is not None:
            output = self.activation(output)
        return output



class DirectInputConnectConditionningMaskedLayer(ConditionningMaskedLayer):
      def __init__(self,
                   units,
                   mask,
                   activation='relu',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros',
                   use_cond_mask=False,
                   direct_mask = None,
                **kwargs):
        self.direct_mask = direct_mask
        super(DirectInputConnectConditionningMaskedLayer, self).__init__(units,
                mask,
                activation,
                kernel_initializer,
                bias_initializer,
                use_cond_mask,
                **kwargs)

      def build(self, input_shape):
        if self.direct_mask is not None:
          self.D = self.add_weight(shape=self.direct_mask.shape,
                                  initializer=self.kernel_initializer,
                                  name='D')
        super().build(input_shape)

      def set_mask(self, mask, direct = False):
        if direct:
          self.direct_mask = mask
        else:
          super().set_mask(mask)

      def get_mask(self, direct = False):
        if direct:
          return self.direct_mask
        else:
          return super().get_mask

      def call(self, inputs):
        if self.direct_mask is None:
          return super().call(inputs)
        input, direct_input = inputs[0], inputs[1]

        masked_w_weights = self.W*self.mask
        weighted_input = k.dot(input, masked_w_weights)
        weighted_input_and_bias = k.bias_add(weighted_input, self.bias, data_format = 'channels_last')
        weighted_direct_input = k.dot(direct_input, self.D * self.direct_mask)

        if self.use_cond_mask:
          masked_u_weights_times_one_vec = k.dot(tf.ones(tf.shape(input)),self.U*self.mask)
          output = weighted_direct_input + weighted_input_and_bias + masked_u_weights_times_one_vec

        else: output = weighted_direct_input + weighted_input_and_bias

        if self.activation is not None:
            output = self.activation(output)
        return output

# MADE Model

In [5]:
# outputs: output Layer   ---------- Both needed when using ----------
# inputs: input Layer     ----------    base keras.Model    ----------     
# mask_generator: Mask Generator instance that manages the Models Masks
# order_agn: Boolean defining if training should be order_agnostic
# conn_agn: Boolean defining if training should be connectivity_agnostic
# direct_input: Boolean defining if direct input masks should be used
class ModelMADE(tf.keras.Model):
    def __init__(self, inputs, outputs, mask_generator, order_agn, conn_agn,
                 #order_agn_step_size, conn_agn_step_size, 
                 direct_input, **kwargs):
      super(ModelMADE, self).__init__(inputs = inputs, outputs = outputs, **kwargs)
      self.mask_generator = mask_generator
      self.order_agn = order_agn
      self.conn_agn = conn_agn
     # self.order_agn_step_size = order_agn_step_size
     # self.conn_agn_step_size = conn_agn_step_size
      self.direct_input = direct_input
     # self.step = 1
    
    # Method called by fit for every batch
    def train_step(self, data):


      # reoder inputs, change masks
      if self.order_agn:
        # order agnostic and connectivity agnostic training
        if self.conn_agn:
          self.mask_generator.shuffle_inputs(return_mask = False)
          new_masks = self.mask_generator.shuffle_masks()
          for hidden_layer_id in range(len(new_masks)):
            self.layers[1+hidden_layer_id].set_mask(new_masks[hidden_layer_id]) #assign layer+1 since the first layer is no hidden layer and has no mask
        
        # order agnostic but not connectivity agnostic training        
        else:
          self.layers[1].set_mask(self.mask_generator.shuffle_inputs())
        if self.direct_input:
          self.layers[-1].set_mask(self.mask_generator.get_direct_mask(), direct=True)

      # not order agnostic but connectivity agnostic training
      elif self.conn_agn:
        new_masks = self.mask_generator.shuffle_masks()
        for hidden_layer_id in range(len(new_masks)):
          self.layers[1+hidden_layer_id].set_mask(new_masks[hidden_layer_id])


      # Unpack the data. Its structure depends on your model and
      # on what you pass to `fit()`.
      x, y = data

    #  self.step += 1

      with tf.GradientTape() as tape:
        y_pred = self(x, training=True)  # Forward pass
        # Compute the loss value
        # (the loss function is configured in `compile()`)
        loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

      

      # Compute gradients
      trainable_vars = self.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)
      # Update weights
      self.optimizer.apply_gradients(zip(gradients, trainable_vars))
      # Update metrics (includes the metric that tracks the loss)
      self.compiled_metrics.update_state(y, y_pred)
      # Return a dict mapping metric names to current value
      return {m.name: m.result() for m in self.metrics}

# MADE Object
responsible for building and inintalizing the MADE model

In [6]:
# units_per_layer = Array containing # of units per layer
# natural_input_order = Boolean defining if the natural input order (x1, x2, x3 etc) should be used
# num_masks: The amount of masks that will be cycled through during training. if num_masks == 1 then connectivity agnostic training is disabled
# order_agn: Boolean defining if training should be order_agnostic
# connectivity_weights: Boolean defining if connectivity weights should be used
# direct input: Boolean defining if there should be a direct input connection between input & output layer
  # seed = The seed used for randomly sampling the masks, for guaranteeing reproducability
class MADE(object):
  def __init__(self, units_per_layer, natural_input_order, num_masks, order_agn,
               #order_agn_step_size, conn_agn_step_size, 
               connectivity_weights, direct_input, seed = "42"):
    self.units_per_layer = units_per_layer
    self.natural_input_order = natural_input_order
    self.num_masks = num_masks
    self.order_agn = order_agn
    #self.order_agn_step_size = order_agn_step_size
    #self.conn_agn_step_size = conn_agn_step_size
    self.connectivity_weights = connectivity_weights
    self.direct_input = direct_input
    self.seed = seed
    self.mask_generator = MaskGenerator(num_masks, units_per_layer, natural_input_order, seed)

  def build_model(self):
    # build input layer
    a = Input(shape = (self.units_per_layer[0],))
    x_layers = []
      
    #build masks
    masks = self.mask_generator.shuffle_masks()
    direct_mask = None

    #build hidden layers  
    for i in range(1,len(self.units_per_layer)-1): #exclude input & output layer
      if i == 1:
        x_layers.append(ConditionningMaskedLayer(units = self.units_per_layer[i], mask = masks[i-1], use_cond_mask = self.connectivity_weights)(a)) #activation is relu, call custom_masking with previous layer as input-param
      else:
        x_layers.append(ConditionningMaskedLayer(units = self.units_per_layer[i], mask = masks[i-1], use_cond_mask = self.connectivity_weights)(x_layers[i-1]))
          
    #build output layer, output layer's activation is sigmoid.
    if self.direct_input:
      direct_mask = self.mask_generator.get_direct_mask()
      output_layer = DirectInputConnectConditionningMaskedLayer(units = self.units_per_layer[-1], mask = masks[-1], activation='sigmoid', use_cond_mask = self.connectivity_weights, direct_mask = direct_mask)([x_layers[-1], a])
    else:
      output_layer = ConditionningMaskedLayer(units = self.units_per_layer[-1], mask = masks[-1], activation='sigmoid', use_cond_mask = self.connectivity_weights)(x_layers[-1])
    x_layers.append(output_layer)
    
    #self.model = Model(inputs = a, outputs = x_layers[-1])
    self.model = ModelMADE(inputs = a, outputs = x_layers[-1], mask_generator = self.mask_generator, order_agn = self.order_agn, conn_agn = self.num_masks>1, 
                           #order_agn_step_size = self.order_agn_step_size, conn_agn_step_size = self.conn_agn_step_size, connectivity_weights=self.connectivity_weights, 
                           direct_input=self.direct_input)
    return self.model

  def summary(self):
    return self.model.summary()

# Loss Function

In [7]:
def cross_entropy_loss(x, x_decoded_mean):
    x = k.flatten(x)
    x_decoded_mean = k.flatten(x_decoded_mean)
    xent_loss = len(X_train[1]) * metrics.binary_crossentropy(x, x_decoded_mean)
    return xent_loss

# Build & Run Model

In [ ]:
######################### Settings #########################
_optimizer_type = "ada" #for any other string here Adagrad is used
_adam_lr = 0.001 #0.1, 0.05, 0.01, 0.005
_ada_lr = 0.01 #0.1, 0.05, 0.01, 0.005
_ada_epsilon = 1e-6

_hidden_layers = [500]
_natural_input_order = False,
_num_masks = 1
_order_agn = True
_order_agn_step_size = 1
_conn_agn_step_size = 1
_connectivity_weights = False
_direct_input = False
_seed = 42
_batch_size = 100
_epochs = 100
_data_portion = 1

if _optimizer_type == "adam": optimizer = Adam(_adam_lr)
else: optimizer = Adagrad(_ada_lr, epsilon = _ada_epsilon)

test_losses = 0
array = []
train_size = int(len(X_train)* _data_portion)
for i in range(10):

  tf.keras.backend.clear_session()

  if _data_portion != 1:
    print(type(train_size))
    idx_shuffled = np.arange(len(X_train))
    random.seed(_seed+i)
    random.shuffle(idx_shuffled)
    X_portion = X_train[idx_shuffled]
    X_portion = X_train[:train_size]
    print(len(X_train))

  else:
    X_portion = X_train

  units_per_layer = np.concatenate(([len(X_train[0])], _hidden_layers, [len(X_train[0])])) #in MADE case the input & output layer have the same amount of units

  temp = MADE(units_per_layer, natural_input_order=_natural_input_order, num_masks = _num_masks, order_agn = _order_agn, 
              #order_agn_step_size = _order_agn_step_size, conn_agn_step_size = _conn_agn_step_size, 
              connectivity_weights = _connectivity_weights, direct_input = _direct_input, seed = _seed)
  model = temp.build_model()
  model.compile(optimizer=optimizer, loss=cross_entropy_loss, run_eagerly=True)

  model.summary()

  start = time.time()

  plt.history = model.fit(
      X_portion, X_portion,
      batch_size=_batch_size,
      epochs=_epochs,
      validation_data=(X_valid, X_valid),
  )
  done = time.time()
  elapsed = done - start
  print("Elapsed: ", elapsed)
  print(f"Number of masks: {_num_masks}")
  test_loss=model.evaluate(X_test, X_test, batch_size=_batch_size)
  print(f"Test Loss: {test_loss}")
  test_losses+=test_loss
  array.append(test_loss)

Model: "model_made"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 conditionning_masked_layer   (None, 500)              392500    
 (ConditionningMaskedLayer)                                      
                                                                 
 conditionning_masked_layer_  (None, 784)              392784    
 1 (ConditionningMaskedLayer                                     
 )                                                               
                                                                 
Total params: 785,284
Trainable params: 785,284
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
500/500 [==============================] - 13s 19ms/step - loss: 183.7745 - val_loss: 137.3862
Epoch 2/1

In [ ]:
  test_loss=model.evaluate(X_test, X_test, batch_size=_batch_size)
  print(f"Test Loss: {test_loss}")

100/100 [==============================] - 1s 7ms/step - loss: 92.9661
Test Loss: 92.96611022949219


In [ ]:
print(array)
print(test_losses/10)
print(np.std(array))
print(np.mean(array))

[101.92285919189453, 101.9369125366211, 102.05867767333984, 102.13648986816406, 101.84507751464844, 101.95286560058594, 101.79920959472656, 101.95958709716797, 101.90636444091797, 101.86487579345703]
101.93829193115235
0.09440509251146503
101.93829193115235


In [ ]:
0.1401634688050654
100.89565582275391
1 mask 

0.09004508236788293
98.4830337524414
2 masks

0.07379556570556475
99.12493896484375
3 masks

0.06985505011918437
99.83944549560547
5 masks

0.09187954082102019
100.64412994384766
10 masks

0.09440509251146503
101.93829193115235
100 masks